In [ ]:
# import necessary libraries
import numpy as np
import pandas as pd

In [5]:
# read the dataset
dataset = pd.read_csv('../data/data.csv')
dataset.head()

,COUNTRY,CODE_TIME,TIME,YEAR,MONTH,MONTH_NAME,PRODUCT,VALUE,DISPLAY_ORDER,yearToDate,previousYearToDate,share
0,Australia,JAN2010,January 2010,2010,1,January,Hydro,990.728,1,16471.891,NaN,0.047771
1,Australia,JAN2010,January 2010,2010,1,January,Wind,409.469,2,4940.909,NaN,0.019744
2,Australia,JAN2010,January 2010,2010,1,January,Solar,49.216,3,908.238,NaN,0.002373
3,Australia,JAN2010,January 2010,2010,1,January,Geothermal,0.083,4,0.996,NaN,0.000004
4,Australia,JAN2010,January 2010,2010,1,January,Total combustible fuels,19289.730,7,214302.969,NaN,0.930108


In [ ]:
# Display dataset information
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 181915 entries, 0 to 181914
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   COUNTRY             181915 non-null  object 
 1   CODE_TIME           181915 non-null  object 
 2   TIME                181915 non-null  object 
 3   YEAR                181915 non-null  int64  
 4   MONTH               181915 non-null  int64  
 5   MONTH_NAME          181915 non-null  object 
 6   PRODUCT             181915 non-null  object 
 7   VALUE               181915 non-null  float64
 8   DISPLAY_ORDER       181915 non-null  int64  
 9   yearToDate          181915 non-null  float64
 10  previousYearToDate  164810 non-null  float64
 11  share               181915 non-null  float64
dtypes: float64(4), int64(3), object(5)
memory usage: 16.7+ MB


In [ ]:
# Display the shape of the dataset
dataset.shape

(181915, 12)

In [11]:
# Define a list of European countries
european_countries = [
    'Austria', 'Belgium', 'Bulgaria', 'Croatia', 'Cyprus', 'Czech Republic',
    'Denmark', 'Estonia', 'Finland', 'France', 'Germany', 'Greece', 'Hungary',
    'Iceland', 'Ireland', 'Italy', 'Latvia', 'Lithuania', 'Luxembourg', 'Malta',
    'Netherlands', 'Norway', 'Poland', 'Portugal', 'Romania', 'Slovakia',
    'Slovenia', 'Spain', 'Sweden', 'Switzerland', 'United Kingdom'
]

# Filter the dataset for European countries
df_europe = dataset[dataset['COUNTRY'].isin(european_countries)]

# Save the filtered data to a new CSV 
df_europe.to_csv("../data/european_countries.csv", index=False)


In [12]:
# preview the first few rows of the filtered dataset
df_europe.head()

,COUNTRY,CODE_TIME,TIME,YEAR,MONTH,MONTH_NAME,PRODUCT,VALUE,DISPLAY_ORDER,yearToDate,previousYearToDate,share
20,Austria,JAN2010,January 2010,2010,1,January,Hydro,2579.264,1,39502.566,NaN,0.437055
21,Austria,JAN2010,January 2010,2010,1,January,Wind,200.966,2,1961.814,NaN,0.034054
22,Austria,JAN2010,January 2010,2010,1,January,Solar,2.864,3,84.421,NaN,0.000485
23,Austria,JAN2010,January 2010,2010,1,January,Geothermal,0.111,4,1.332,NaN,0.000019
24,Austria,JAN2010,January 2010,2010,1,January,Total combustible fuels,3117.160,7,26042.221,NaN,0.528201


In [14]:
# write name of the column in lowercase
df_europe.columns = df_europe.columns.str.lower()
df_europe.head()

,country,code_time,time,year,month,month_name,product,value,display_order,yeartodate,previousyeartodate,share
20,Austria,JAN2010,January 2010,2010,1,January,Hydro,2579.264,1,39502.566,NaN,0.437055
21,Austria,JAN2010,January 2010,2010,1,January,Wind,200.966,2,1961.814,NaN,0.034054
22,Austria,JAN2010,January 2010,2010,1,January,Solar,2.864,3,84.421,NaN,0.000485
23,Austria,JAN2010,January 2010,2010,1,January,Geothermal,0.111,4,1.332,NaN,0.000019
24,Austria,JAN2010,January 2010,2010,1,January,Total combustible fuels,3117.160,7,26042.221,NaN,0.528201
